In [395]:
from __future__ import print_function

In [396]:
import csv
from csv import reader
import math
import random 
from random import randint, seed 
from math import sqrt

In [397]:
def readData(filename):
    with open(filename, 'rt') as read_obj:
        csv_reader = csv.reader(read_obj) # Return a reader object which will
                                        # iterate over lines in the given csvfile
        training_data = list(csv_reader)
    return training_data

In [398]:
def convert_to_float(row):  
    row = [float(x.strip()) for x in row]
    return row

In [399]:
def class_counts(data_set):
    counts = {}
    for row in data_set:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [400]:
def calculate_entropy(data_set):
    counts = class_counts(data_set)
    entropy = 0
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(data_set))
        entropy -= math.log(prob_of_lbl, 2)
    return entropy

In [401]:
class Question:

    def __init__(self, index, value):
        self.index = index
        self.value = value

    def match(self, example):
        val = example[self.index]
        return val >= self.value

In [402]:
def make_partition(data_set, question):
    right, left = [], []
    for row in data_set:
        if question.match(row):
            right.append(row)
        else:
            left.append(row)
    return right, left

In [403]:
def calculate_gain(right, left, current_data_set_entropy):
    p = float(len(left)) / (len(left) + len(right))
    return current_data_set_entropy - p * calculate_entropy(left) - (1 - p) * calculate_entropy(right)

In [404]:
def find_best_split(data_set):
    best_gain = 0
    split_question = None
    current_data_set_entropy = calculate_entropy(data_set)
    number_of_features = len(data_set[0]) - 1

    for column in range(number_of_features):
        column_values = set([row[column] for row in data_set])
        for value in column_values:
            question = Question(column, value)
            right, left = make_partition(data_set, question)
            if len(right) == 0 or len(left) == 0:
                continue

            gain = calculate_gain(right, left, current_data_set_entropy)
            
            if gain > best_gain:
                best_gain, split_question = gain, question
    
    return best_gain, split_question

In [405]:
class leaf_node:
    def __init__(self, data_set):
       self.predictions = class_counts(data_set)

In [406]:
class decision_node:
    def __init__(self, question, right, left):
        self.question = question
        self.right = right
        self.left = left

In [407]:
def build_tree(data_set):
    info_gain, question = find_best_split(data_set)

    if info_gain == 0:
        return leaf_node(data_set)
    
    right_split_data_set, left_split_data_set = make_partition(data_set, question)
    right_branch = build_tree(right_split_data_set)
    left_branch = build_tree(left_split_data_set)
    return decision_node(question, right_branch, left_branch) 


In [408]:
def print_tree(node, spacing=""):
    if isinstance(node, leaf_node):
        print(spacing + "Node class and count: ", node.predictions)
        return
    print(spacing + 'index: ' + str(node.question.index) +
          ' value: ' + str(node.question.value))

    print(spacing + '--> greater than:')
    print_tree(node.right, spacing + "  ")

    print(spacing + '--> less than:')
    print_tree(node.left, spacing + "  ")

In [409]:
def classify(row, node):

    if isinstance(node, leaf_node):
        return node.predictions

    if node.question.match(row):
        return classify(row, node.right)
    else:
        return classify(row, node.left)

In [410]:
#### Main ####
initial_data_set = readData('wine.csv') # here dataset contains data values as strings
#so we convert the string values to floats
data_set = []
for row in initial_data_set:
    row = convert_to_float(row)
    data_set.append(row)
 
# now we will do k-fold 
# for now k=10 
k = 10
folds = []
for i in range(k):
    folds.append([])
for i in range(len(data_set)):
    folds[i % k].append(data_set[i])

# now we will do cross validation
total_accuracy = 0.0
for group in folds:
    # train test splits
    train_data = list(folds)
    train_data.remove(group)
    train_data = sum(train_data, [])
    test_data = group  


    # let we will make 5 decision trees
    num_of_decision_tree = 5
    n_features = int(sqrt(len(train_data[0])-1))
    trees = list()

    # here we will build forest trees and aslo test the dataset
    # first build trees and then test the rows 
    total_row = 0
    total_matched = 0
    for t_row in test_data:
        total_row += 1
        forest_matched = 0
        forest_unmatched = 0
        for i in range(num_of_decision_tree):
            # get the number of rows and columns in the data
            num_rows = len(train_data)
            num_cols = len(train_data[0])-1   # to avoid taking feature column we -1

            # create a list of row indices and randomly shuffle it
            row_indices = list(range(num_rows))
            random.shuffle(row_indices)

            # take 50% of rows
            num_tree_rows = int(0.5 * num_rows)
            ith_tree_rows_indices = row_indices[:num_tree_rows]

            # columns
            ith_tree_cols_indices = random.sample(range(num_cols), n_features)

            # now our ith tree train_dataset
            # extract the last column
            feature_column = []
            for i in ith_tree_rows_indices:
                feature_column.append(train_data[i][-1])

            ith_train_set = [[train_data[i][j] for j in ith_tree_cols_indices] for i in ith_tree_rows_indices]
            for row in ith_train_set:
                row.append(feature_column[ith_train_set.index(row)])

            # now build tree
            ith_d_tree = build_tree(ith_train_set)
            trees.append(ith_d_tree)  # if you want you can print the trees later from the tree list

            # now ith-test data
            # here we done a very significant work
            # we take the test_row accorind to training data column indexes
            # if we don't take it it will show error answers
            test_row = []
            for index in ith_tree_cols_indices:
                value = t_row[index]
                test_row.append(value)
            test_row.append(t_row[-1])
            
            # now test the row for this ith-d_tree
            classified = classify(test_row, ith_d_tree)
            for lbl in classified.keys():
                if lbl == test_row[-1]:
                    forest_matched += 1
                else:
                    forest_unmatched += 1

        # we take the maximum decision form the trees
        if forest_matched > forest_unmatched:
            total_matched += 1

    accuracy = total_matched/total_row*100
    total_accuracy += accuracy
    print('accurary: ', accuracy, '%')


print("\n*********Final Accuracy********\n")
average_accuracy = total_accuracy / k
print('average accuracy: ', average_accuracy, "%")

accurary:  100.0 %
accurary:  88.88888888888889 %
accurary:  94.44444444444444 %
accurary:  88.23529411764706 %
accurary:  88.23529411764706 %
accurary:  94.11764705882352 %
accurary:  100.0 %
accurary:  82.35294117647058 %
accurary:  88.23529411764706 %
accurary:  82.35294117647058 %

*********Final Accuracy********

average accuracy:  90.68627450980392 %
